In [3]:
import re
import os
import pandas as pd
import string


C:\Users\Brian\AppData\Local\Temp\ipykernel_22192\3211376276.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Import files Contents

In [49]:
objects = [
    "Table",
    "Page",
    "Report",
    "Codeunit",
    "Query",
    "XMLport"
]

# Tables
# with open("src/Tables.txt", "r") as tbl_file:
#     tables = tbl_file.read()
#     print("Imported tables: ", len(tables))
''
# Pages
# with open("src/Pages.txt", "r") as pg_file:
#     # pages = pg_file.read()
#     tables = pg_file.read()
#     print("Imported Pages: ", len(tables))


# Reports
# with open("src/Reports.txt", "r") as rpt_file:
#     # reports = rpt_file.read()
#     tables = rpt_file.read()
#     print("Imported reports: ", len(tables))


# Codeunit
# with open("src/Codeunits.txt", "r") as cu_file:
#     tables = cu_file.read()
#     # codeunits = cu_file.read()
#     print("Imported codeunits: ", len(tables))


# # Queries
with open("src/Queries.txt", "r") as qry_file:
    tables = qry_file.read()
    # queries = qry_file.read()
    print("Imported Queries: ", len(tables))
    

OBJECT = "Query"

Imported Queries:  15475


In [50]:
# find table objects in tables file. (OBJECT Table 50000 Bank Code\n{)
ptn = f"\nOBJECT {OBJECT} [0-9]+ .+\n"
found_tables = re.findall(ptn, tables)
found_tables = [table[:-1].strip() for table in found_tables]
print("found: ", len(found_tables))



# generate usable meta-data
table_meta_data = []
for table in found_tables:
    start = tables.index(table)
    end = start + len(table)
    
    tbl_id = re.findall("[0-9]+", table)[0]
    tbl_name = table.split(tbl_id)[-1].strip()
    table_meta_data.append([(start, end), int(tbl_id), tbl_name, table])

print(f"cleaned: {len(table_meta_data)}\nSAMPLE: ", table_meta_data[-1])


found:  5
cleaned: 5
SAMPLE:  [(14439, 14475), 50183, 'Portal Documents1', 'OBJECT Query 50183 Portal Documents1']


In [51]:
# Change the object ID
def gen_id(old_id:int):
    return (old_id - 50000) + 700000

def gen_table_def(table):
    _, tbl_id, tbl_name, _ = table
    return f"OBJECT {OBJECT} {gen_id(tbl_id)} {tbl_name}"


for idx, table in enumerate(table_meta_data):
    _, tbl_id, tbl_name, search_str = table
    
    # Dont use the index because the tables changes with each iteration.
    start = tables.index(search_str)
    end = start + len(search_str)
    
    new_table_def = gen_table_def(table)

    # This is broken
    # tables = tables.replace(old_table_def, new_table_def, 1)
    tables = tables[:start] + new_table_def + tables[end:]


with open("task.txt", "w") as f:
    f.write(tables)

print("Table ids have been updated.")
# A this point rerun the previous two cells - reset the table_meta_data with new data. 

Table ids have been updated.


In [52]:
# find table objects in tables file. (OBJECT Table 50000 Bank Code\n{)
ptn = f"\nOBJECT {OBJECT} [0-9]+ .+\n"
found_tables = re.findall(ptn, tables)
found_tables = [table[:-1].strip() for table in found_tables]
print("found: ", len(found_tables))


# generate usable meta-data
table_meta_data = []
for table in found_tables:
    start = tables.index(table)
    end = start + len(table)

    tbl_id = re.findall("[0-9]+", table)[0]
    tbl_name = table.split(tbl_id)[-1].strip()
    table_meta_data.append([(start, end), int(tbl_id), tbl_name, table])

print(f"cleaned: {len(table_meta_data)}\nSAMPLE: ", table_meta_data[-1])

print("updated tables meta data list ")

found:  5
cleaned: 5
SAMPLE:  [(14443, 14480), 700183, 'Portal Documents1', 'OBJECT Query 700183 Portal Documents1']
updated tables meta data list 


In [53]:

# with open("task.txt", "r") as f:
#     tables = f.read()

# Find the object Contents 
sections = []
for idx, table in enumerate(table_meta_data):
    search_str = table[-1]
    start = tables.index(search_str) + len(search_str)

    sections.append([start+1]) 
    if(idx > 0):
        sections[-2] = [*sections[-2], start - len(search_str) - 2]

print("Found objects contents: ", len(sections))
sections[-1] = [*sections[-1], -1]


Found objects contents:  5


In [54]:
# Save out the table objects.

for section, meta_data in zip(sections, table_meta_data):
    start, end = section
    _, table_id, table_name, obj_def = meta_data

    for char in string.punctuation:
        table_name = table_name.replace(char, "")

    with open(f"temp/{OBJECT}/{table_id} - {table_name}.txt", "w") as f: 
        contents = f"{obj_def}\n" + tables[start:end]
        f.write(contents)

print("Save the Tables")

Save the Tables


In [55]:
def gen_object_id(obj):
    if (" " not in obj):
        obj_id = re.findall("[0-9]+", obj)[-1]
        obj_type = "".join(re.findall(r"\D", obj))
        obj = f"{obj_type} {obj_id}"
        obj = gen_object_id(obj)
        return obj.replace(" ", "")
    
    obj_type, old_id = obj.split(" ")
    old_id = int(old_id)
    if(old_id >= 50000 and old_id <=80000):
        new_id = (old_id - 50000) + 700000
    else:
        new_id = old_id
    return f"{obj_type} {new_id}"
print("Record 50000 -->", gen_object_id("Record 50000"))
print("Table50000 -->", gen_object_id("Table50000"))

Record 50000 --> Record 700000
Table50000 --> Table700000


In [56]:
table_files = os.listdir(f"temp/{OBJECT}")
table_files = [f"temp/{OBJECT}/{file}" for file in table_files ]

for table_file in table_files:
    with open(f"{table_file}", "r") as f:
        contents = f.read()

    imported_tables = list(set(re.findall("Table[0-9]+", contents)))
    imported_tables = [(obj, gen_object_id(obj)) for obj in imported_tables]

    imported_records =list( set(re.findall("Record [0-9]+", contents)))
    imported_records = [(obj, gen_object_id(obj)) for obj in imported_records]
    
    imported_pages =list( set(re.findall("Page [0-9]+", contents)))
    imported_pages = [(obj, gen_object_id(obj)) for obj in imported_pages]
    
    imported_reports =list( set(re.findall("Report [0-9]+", contents)))
    imported_reports = [(obj, gen_object_id(obj)) for obj in imported_reports]
    
    imported_code_units =list( set(re.findall("Codeunit [0-9]+", contents)))
    imported_code_units = [(obj, gen_object_id(obj)) for obj in imported_code_units]
    
    imported_XMLports =list( set(re.findall("XMLport [0-9]+", contents)))
    imported_XMLports = [(obj, gen_object_id(obj)) for obj in imported_XMLports]
    
    imported_queries =list( set(re.findall("Query [0-9]+", contents)))
    imported_queries = [(obj, gen_object_id(obj)) for obj in imported_queries]
    
    for imported_obj in [imported_tables, imported_records, imported_pages, imported_reports, imported_code_units, imported_XMLports,  imported_queries ]:
        for old_obj, new_obj in imported_obj:
            contents = re.sub(old_obj, new_obj, contents)

    with open(f"{table_file}", "w") as f:
        f.write(contents)

In [57]:
re.sub("br", "brian", "br a br")

'brian a brian'

In [100]:
imported_objects = []

for section in sections:
    content = tables[section[0]:section[-1]]

    imported_tables = list(set(re.findall("Table[0-9]+", content)))
    imported_records =list( set(re.findall("Record [0-9]+", content)))
    imported_pages =list( set(re.findall("Page [0-9]+", content)))
    imported_reports =list( set(re.findall("Report [0-9]+", content)))
    imported_code_units =list( set(re.findall("Codeunit [0-9]+", content)))
    imported_XMLports =list( set(re.findall("XMLport [0-9]+", content)))
    imported_queries =list( set(re.findall("Query [0-9]+", content)))

    imported_objects.append([
        [*imported_tables, *imported_records], 
        imported_pages,
        imported_reports, 
        imported_code_units, 
        imported_XMLports, 
        imported_queries
    ])


print("Located impored objects within the tables")

Located impored objects within the tables


In [27]:
# Change the imported object IDs


    
def gen_object_id(obj):
    if (" " not in obj):
        obj_id = re.findall("[0-9]+", obj)[-1]
        obj_type = "".join(re.findall(r"\D", obj))
        obj = f"{obj_type} {obj_id}"
        obj = gen_object_id(obj)
        return obj.replace(" ", "")
    
    obj_type, old_id = obj.split(" ")
    old_id = int(old_id)
    if(old_id >= 50000 and old_id <=80000):
        new_id = (old_id - 50000) + 700000
    else:
        new_id = old_id
    return f"{obj_type} {new_id}"


gen_object_id("Record 50001")

'Record 700001'

In [102]:

for section, table_objects in zip(sections, imported_objects):
    for objects_list in table_objects:
        if objects_list:
            for obj in objects_list:
                start, end = section
                content = tables[start:end]
                new_obj = gen_object_id(obj)
                content = content.replace(obj, new_obj)
                with open(f"temp/start - {start}.txt", "w") as f:
                    f.write(content )
                print(obj, new_obj,  content.find(obj))  

                tables = tables[:start] +  content + tables[end:]          


Record 50000 Record 700000 -1
Record 50008 Record 700008 -1
Record 15 Record 15 32907
Record 50002 Record 700002 -1
Record 50003 Record 700003 -1
Record 50076 Record 700076 -1
Record 5600 Record 5600 33043
Record 270 Record 270 32943
Record 18 Record 18 32977
Record 50027 Record 700027 -1
Record 50088 Record 700088 -1
Record 5200 Record 5200 33161
Record 23 Record 23 33008
Record 50031 Record 700031 -1
Record 50013 Record 700013 -1
Record 50024 Record 700024 -1
Record 50025 Record 700025 -1
Record 330 Record 330 33283
Record 454 Record 454 34505
Record 271 Record 271 33128
Record 413 Record 413 33080
Record 50010 Record 700010 -1
Record 50012 Record 700012 -1
Codeunit 396 Codeunit 396 32868
Codeunit 50000 Codeunit 700000 -1
Record 413 Record 413 91357
Record 50027 Record 700027 -1
Record 15 Record 15 91186
Record 50008 Record 700008 -1
Record 25 Record 25 42100
Record 330 Record 330 91551
Record 50002 Record 700002 -1
Record 51171 Record 701171 -1
Record 50003 Record 700003 -1
Record 5

In [103]:
for section, table_objects in zip(sections, imported_objects):
    for objects_list in table_objects:
        if objects_list:
            for obj in objects_list:
                new_obj = gen_object_id(obj)
                print(obj,  new_obj)


with open("task.txt", "w") as f:
    f.write(tables)


Record 50000 Record 700000
Record 50008 Record 700008
Record 15 Record 15
Record 50002 Record 700002
Record 50003 Record 700003
Record 50076 Record 700076
Record 5600 Record 5600
Record 270 Record 270
Record 18 Record 18
Record 50027 Record 700027
Record 50088 Record 700088
Record 5200 Record 5200
Record 23 Record 23
Record 50031 Record 700031
Record 50013 Record 700013
Record 50024 Record 700024
Record 50025 Record 700025
Record 330 Record 330
Record 454 Record 454
Record 271 Record 271
Record 413 Record 413
Record 50010 Record 700010
Record 50012 Record 700012
Codeunit 396 Codeunit 396
Codeunit 50000 Codeunit 700000
Record 413 Record 413
Record 50027 Record 700027
Record 15 Record 15
Record 50008 Record 700008
Record 25 Record 25
Record 330 Record 330
Record 50002 Record 700002
Record 51171 Record 701171
Record 50003 Record 700003
Record 5600 Record 5600
Record 454 Record 454
Record 5200 Record 5200
Record 23 Record 23
Record 122 Record 122
Record 50028 Record 700028
Record 270 Recor

In [104]:
a = "a br a br"
find = "br"
rep = "brian"

idx = a.find(find)
a[0:idx] + rep + a[idx + len(find):]


'a brian a br'

In [105]:
"bbb".replace('b', "blue")

'blueblueblue'

'asdf'